In [ ]:
formula2 = f'''ln_real_price ~ year2005_delta_tax_eff + year2006_delta_tax_eff + year2007_delta_tax_eff + year2008_delta_tax_eff + ln_prop_value + C(year) + C(kommune_old)'''
formula2

'ln_real_price ~ year2005_delta_tax_eff + year2006_delta_tax_eff + year2007_delta_tax_eff + year2008_delta_tax_eff + ln_prop_value + C(year) + C(kommune_old)'

In [ ]:
mod_quantile_fe_within = smf.quantreg(formula2,
              data=df_merged_index)
result_quantile=mod_quantile_fe_within.fit(q=0.5,cov_type='cluster', cov_kwds={'groups': df_merged_index['kommune_old']}, fe=True)
result_quantile.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                         QuantReg Regression Results                          
==============================================================================
Dep. Variable:          ln_real_price   Pseudo R-squared:               0.5060
Model:                       QuantReg   Bandwidth:                     0.01402
Method:                 Least Squares   Sparsity:                       0.4851
Date:                Fri, 14 Apr 2023   No. Observations:               425792
Time:                        14:48:15   Df Residuals:                   425535
                                        Df Model:                          256
==================================================================================================================
                                                     coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                          2.6546      0.012    222.562      0.000       2.631       2.678
C(year)[T.2001]                                -8.792e-05      0.002     -0.053      0.957      -0.003       0.003
C(year)[T.2002]                                   -0.0021      0.002     -1.287      0.198      -0.005       0.001
C(year)[T.2003]                                    0.0018      0.002      1.074      0.283      -0.001       0.005
C(year)[T.2004]                                   -0.0086      0.002     -5.231      0.000      -0.012      -0.005
C(year)[T.2005]                                   -0.0147      0.002     -9.262      0.000      -0.018      -0.012
C(year)[T.2006]                                   -0.0368      0.002    -22.427      0.000      -0.040      -0.034
C(year)[T.2007]                                   -0.0338      0.002    -20.371      0.000      -0.037      -0.031
C(year)[T.2008]                                   -0.0317      0.002    -17.546      0.000      -0.035      -0.028
C(year)[T.2009]                                   -0.0332      0.002    -17.078      0.000      -0.037      -0.029
C(year)[T.2010]                                   -0.0481      0.002    -25.758      0.000      -0.052      -0.044
C(kommune_old)[T.Assens Kommune]                  -0.5342      0.009    -61.680      0.000      -0.551      -0.517
C(kommune_old)[T.Augustenborg Kommune]            -0.5663      0.010    -57.157      0.000      -0.586      -0.547
C(kommune_old)[T.Aulum-Haderup Kommune]           -0.4630      0.010    -47.829      0.000      -0.482      -0.444
C(kommune_old)[T.Ballerup-Måløv Kommune]           0.0023      0.007      0.311      0.755      -0.012       0.017
C(kommune_old)[T.Billund Kommune]                 -0.5013      0.009    -53.019      0.000      -0.520      -0.483
C(kommune_old)[T.Birkerød Kommune]                 0.0779      0.008      9.864      0.000       0.062       0.093
C(kommune_old)[T.Bjergsted Kommune]               -0.2446      0.009    -27.883      0.000      -0.262      -0.227
C(kommune_old)[T.Bjerringbro Kommune]             -0.4700      0.008    -58.756      0.000      -0.486      -0.454
C(kommune_old)[T.Blåbjerg Kommune]                -0.5628      0.010    -57.988      0.000      -0.582      -0.544
C(kommune_old)[T.Blåvandshuk Kommune]             -0.4541      0.012    -38.494      0.000      -0.477      -0.431
C(kommune_old)[T.Bogense Kommune]                 -0.5098      0.011    -47.932      0.000      -0.531      -0.489
C(kommune_old)[T.Bornholms Regionskommune]        -0.1994      0.006    -31.028      0.000      -0.212      -0.187
C(kommune_old)[T.Bov Kommune]                     -0.5696      0.009    -65.062      0.000      -0.587      -0.552
C(kommune_old)[T.Bramming Kommune]                -0.4974      0.008    -63.638      0.000      -0.513      -0.482
C(kommune_old)[T.Bramsnæs Kommune]                -0.2449      0.009    -26.899      0.000      -0.2

In [ ]:
quantiles = np.arange(0.1, 0.9+0.1, 0.1)


def fit_model(q):
    result_quantile = mod_quantile_fe_within.fit(q=q, cov_type='cluster', cov_kwds={'groups': df_merged_index['kommune_old']})
    return [q, result_quantile.params["Intercept"], result_quantile.params["year2007_delta_tax_eff"]] + result_quantile.conf_int().loc[
        "year2007_delta_tax_eff"
    ].tolist()


models_fe_within = [fit_model(x) for x in quantiles]
models_fe_within = pd.DataFrame(models_fe_within, columns=["q","Intercept", "year2007_delta_tax_eff", "lb", "ub"])
models_fe_within

,q,Intercept,year2007_delta_tax_eff,lb,ub
0,0.1,3.645061,-0.006669,-0.022844,0.009507
1,0.2,2.453726,-0.006801,-0.012214,-0.001388
2,0.3,2.287272,-0.015876,-0.018924,-0.012827
3,0.4,2.420242,-0.022956,-0.025250,-0.020662
4,0.5,2.654561,-0.025829,-0.027692,-0.023967
5,0.6,2.908082,-0.028811,-0.030492,-0.027131
6,0.7,3.224490,-0.030126,-0.031792,-0.028460
7,0.8,3.652365,-0.036658,-0.038561,-0.034755
8,0.9,4.500499,-0.050423,-0.053415,-0.047431


In [ ]:
models_fe_within.to_parquet('models/quantiles_fe_within.pq')